In [2]:
import numpy as np
import cupy as cp
import ctypes
from cupy.cuda import memory
from cuquantum import custatevec as cusv
from cuquantum import cudaDataType as cudtype

# 一、单cuda流

cuQuantum-python API 包括 cuStateVec 和 cuTensorNet，分别用于处理状态向量以及张量网络。
与 C/C++ 不同，Python 不提供用于分配/释放主机内存（更不用说设备内存）的低级原语，对Python 代理对象正确进行内存管理非常重要。

## 1.1内存管理

cuQuantum Python要求用户使用 NumPy（用于主机内存）和 CuPy（用于设备内存）来满足此类需求。

如何使用 NumPy 和 CuPy

在主机内存中存储数据（NumPy,CuPy）：

In [142]:
# 创建一个NumPy数组，存储在CPU内存中
host_array = np.array([1.0, 2.0, 3.0])
# 创建一个CuPy数组，存储在GPU内存中
device_array = cp.array([1.0, 2.0, 3.0])

数据在主机内存与设备内存间转移

In [143]:
# 将NumPy数组从CPU内存传输到GPU内存
device_array = cp.asarray(host_array)
# 将CuPy数组从GPU内存传输到CPU内存
host_array_back = cp.asnumpy(device_array)

使用 NumPy 和 CuPy 来创建主机（CPU）内存和设备（GPU）内存的缓冲区，并将它们传递给外部函数（my_func），该函数可以在 GPU 上或 CPU 上修改这些缓冲区的数据。

In [144]:
# 定义一个简单的函数，模拟修改传入的缓冲区
# 如果是主机内存，则乘以10，如果是设备内存，则加5
def my_func(ptr, size, is_device=False):
    if is_device:
         # 如果是设备内存，假设传入的是一个设备内存指针
        # 使用 cp.ndarray 并创建一个 MemoryPointer 对

        buf = cp.ndarray(shape=(size,), dtype=cp.float64, memptr=ptr)  # 通过 memptr 传递指针
        buf += 5  # 对设备内存中的每个元素加 5
    else:
        # 如果是主机内存，通过 ctypes 来操作
        # 假设传入的是主机内存缓冲区的指针
        buf = np.ctypeslib.as_array(ctypes.cast(ptr, ctypes.POINTER(ctypes.c_int32)), shape=(size,))
        buf *= 10  # 对主机内存中的每个元素乘以 10

In [145]:
# === 处理主机内存（NumPy） ===
# 创建一个主机内存缓冲区，大小为 5，类型为 int32
buf = np.empty((5,), dtype=np.int32)
buf[:] = [1, 2, 3, 4, 5]  # 初始化缓冲区内容
print("Before modifying host buffer:", buf)
# 通过 ctypes 获取 NumPy 数组的指针
my_func(buf.ctypes.data, buf.size, is_device=False)
print("After modifying host buffer:", buf)

Before modifying host buffer: [1 2 3 4 5]
After modifying host buffer: [10 20 30 40 50]


从 cuQuantum Python v0.1.0 开始,指针地址只是简单地转换为纯 int以便传递。如果需要，用户可以完全控制（并负责）管理指针生命周期。

cuQuantum从v22.03开始,为用户提供了一个接口，用于将内存池引入 cuStateVec/cuTensorNet 库以供使用。
用户在调用 API 之前不再需要管理任何临时工作区,库将从用户的池中提取内存（并在完成后将其返回）。

## 1.2 example-Toffi门

1) 导入必要的库

In [181]:
import numpy as np
import cupy as cp
from cuquantum import custatevec as cusv
from cuquantum import cudaDataType as cudtype
from cuquantum import ComputeType as ctype

numpy：用于处理数组和数值计算。\
cupy：用于处理 GPU 加速的数组，这是 CuPy 库，它能在 NVIDIA GPU 上运行与 NumPy 类似的代码。\
cuquantum：一个用于量子计算的库，提供了与 cuStateVec 和 cuTensorNet 库的接口，用于量子计算的加速计算。这里我们主要使用 custatevec 来处理量子态向量。\
cudaDataType 和 ComputeType：这些导入了用于指定 CUDA 数据类型和计算类型的枚举，帮助指定在 CUDA 上进行计算时使用的数据类型和计算精度。

2) 初始化参数

In [190]:
nIndexBits = 3
nSvSize = (1 << nIndexBits)
nTargets = 1
nControls = 2
adjoint = 0

targets = (2,)
controls = (0, 1)

nIndexBits：表示量子态向量的位数，3 位表示有 8 个量子比特。\
nSvSize：计算量子态向量的大小，1 << nIndexBits 等价于 2^3 = 8，因此有 8 个元素的量子态向量。\
nTargets：应用的目标量子比特的数量，这里是 1。\
nControls：应用的控制量子比特的数量，这里是 2。\
adjoint：表示是否计算矩阵的伴随矩阵，0 表示不计算伴随矩阵。\
targets 和 controls：分别是目标量子比特和控制量子比特的索引。

3) 初始化量子态向量和矩阵

In [183]:
d_sv = cp.asarray([[0.0, 0.0], [0.0, 0.1], [0.1, 0.1], [0.1, 0.2],
                   [0.2, 0.2], [0.3, 0.3], [0.3, 0.4], [0.4, 0.5]], dtype=np.float64)
d_sv = d_sv.view(np.complex128).reshape(-1)

d_sv_result = cp.asarray([[0.0, 0.0], [0.0, 0.1], [0.1, 0.1], [0.4, 0.5],
                          [0.2, 0.2], [0.3, 0.3], [0.3, 0.4], [0.1, 0.2]], dtype=np.float64)
d_sv_result = d_sv_result.view(np.complex128).reshape(-1)

d_matrix = cp.asarray([[0.0, 0.0], [1.0, 0.0], [1.0, 0.0], [0.0, 0.0]], dtype=np.float64)
d_matrix = d_matrix.view(np.complex128).reshape(-1)

d_sv：表示一个 量子态向量，它是一个二维数组，初始化为一个复杂的浮动矩阵，且通过 .view(np.complex128) 使得其数据类型为 complex128。\
d_sv_result：是预期的结果量子态向量，用于验证计算结果是否正确。\
d_matrix：表示一个 量子门矩阵，例如一个单量子比特门，它也是一个 complex128 数据类型的数组。

4) 初始化 cuStateVec 句柄

In [184]:
handle = cusv.create()

cusv.create()：该函数创建一个 cuStateVec 句柄，这个句柄在后续的 API 调用中用于执行量子计算任务。

5) 计算外部工作区所需的内存大小

In [185]:
extraWorkspaceSizeInBytes = cusv.apply_matrix_get_workspace_size(
    handle, cudtype.CUDA_C_64F, nIndexBits, d_matrix.data.ptr, cudtype.CUDA_C_64F,
    cusv.MatrixLayout.ROW, adjoint, nTargets, nControls, ctype.COMPUTE_64F)

cusv.apply_matrix_get_workspace_size()：该函数计算在应用量子门矩阵时所需的外部工作区内存的大小。返回值是所需的内存大小，以字节为单位。\
参数解释：\
handle：句柄，表示量子计算的上下文。\
cudtype.CUDA_C_64F：指定数据类型，这里是 64 位浮点数。\
nIndexBits：量子比特数，决定量子态的大小。\
d_matrix.data.ptr：量子门矩阵的指针，表示在内存中的位置。\
cusv.MatrixLayout.ROW：矩阵布局，指定行优先排列。\
adjoint：是否计算矩阵的伴随矩阵。\
nTargets 和 nControls：目标量子比特和控制量子比特的数量。\
ctype.COMPUTE_64F：指定计算使用的精度类型。

6) 如果需要，分配外部工作区内存

In [186]:
if extraWorkspaceSizeInBytes > 0:
    workspace = cp.cuda.alloc(extraWorkspaceSizeInBytes)
    workspace_ptr = workspace.ptr
else:
    workspace_ptr = 0

外部工作区：如果需要的内存大小大于 0，则使用 CuPy 分配设备内存（cp.cuda.alloc()）并返回指针。否则，工作区指针为 0，表示不需要额外内存。

7) 应用量子门矩阵到量子态向量

In [187]:
cusv.apply_matrix(
    handle, d_sv.data.ptr, cudtype.CUDA_C_64F, nIndexBits,
    d_matrix.data.ptr, cudtype.CUDA_C_64F, cusv.MatrixLayout.ROW, adjoint,
    targets, len(targets), controls, 0, len(controls), ctype.COMPUTE_64F,
    workspace_ptr, extraWorkspaceSizeInBytes)


cusv.apply_matrix()：该函数应用量子门矩阵到量子态向量。它使用 cuStateVec API 将量子门矩阵应用到量子态，并通过工作区执行计算。\
参数：\
handle：量子计算上下文。\
d_sv.data.ptr：量子态向量的指针。\
d_matrix.data.ptr：量子门矩阵的指针。\
nIndexBits：量子比特数。\
targets 和 controls：目标量子比特和控制量子比特的索引。\
workspace_ptr：工作区指针，指向分配的内存空间。\
extraWorkspaceSizeInBytes：所需工作区大小。

8) 销毁句柄

In [188]:
cusv.destroy(handle)

cusv.destroy()：销毁 cuStateVec 句柄，释放相关资源。

9)  检查计算结果是否正确

In [189]:
correct = cp.allclose(d_sv, d_sv_result)
if not correct:
    raise RuntimeError("example FAILED: wrong result")

cp.allclose()：检查计算后的量子态向量 d_sv 是否与预期结果 d_sv_result 相同。
如果结果不匹配，则抛出异常，表示计算失败。

# 二、多Cuda流

## 2.1 Cuda流

大多数 cuStateVec API 的执行都是在附加到 创建的 cuStateVec 句柄的流上序列化的 初始流是默认流。\
用户可以通过调用 句柄custatevecSetStream() 将用户创建的流设置为 cuStateVec \
所有类型的流（默认、阻塞和非阻塞）都是可以接受的。

In [6]:
import cupy as cp
from cuquantum import custatevec as cusv
from cuquantum import cudaDataType as cudtype
from cuquantum import ComputeType as ctype

# 参数配置
nIndexBits = 3  # 3 个量子比特
adjoint = 0  # 是否使用伴随矩阵
targets = (2,)  # 目标量子比特
controls = (0, 1)  # 控制量子比特

# 创建两个自定义 CUDA 流
stream1 = cp.cuda.Stream()  # 流1
stream2 = cp.cuda.Stream()  # 流2

# 为每个流创建一个 cuStateVec 上下文句柄
handle1 = cusv.create()
handle2 = cusv.create()

# 将每个句柄绑定到不同的流
cusv.set_stream(handle1, stream1.ptr)
cusv.set_stream(handle2, stream2.ptr)

# 为每个流分配独立的状态向量和量子门矩阵
d_sv1 = cp.asarray([[0.0, 0.0], [0.0, 0.1], [0.1, 0.1], [0.1, 0.2],
                    [0.2, 0.2], [0.3, 0.3], [0.3, 0.4], [0.4, 0.5]], dtype=cp.float64).view(cp.complex128)
d_sv2 = cp.asarray([[0.2, 0.2], [0.3, 0.3], [0.3, 0.4], [0.4, 0.5],
                    [0.1, 0.1], [0.2, 0.3], [0.4, 0.4], [0.5, 0.6]], dtype=cp.float64).view(cp.complex128)

d_matrix = cp.asarray([[0.0, 0.0], [1.0, 0.0], [1.0, 0.0], [0.0, 0.0]], dtype=cp.float64).view(cp.complex128)

# 获取每个流所需的工作区大小，并分配内存
extraWorkspaceSizeInBytes1 = cusv.apply_matrix_get_workspace_size(
    handle1, cudtype.CUDA_C_64F, nIndexBits, d_matrix.data.ptr, cudtype.CUDA_C_64F,
    cusv.MatrixLayout.ROW, adjoint, len(targets), len(controls), ctype.COMPUTE_64F)

extraWorkspaceSizeInBytes2 = cusv.apply_matrix_get_workspace_size(
    handle2, cudtype.CUDA_C_64F, nIndexBits, d_matrix.data.ptr, cudtype.CUDA_C_64F,
    cusv.MatrixLayout.ROW, adjoint, len(targets), len(controls), ctype.COMPUTE_64F)

# 分配流1的工作区
if extraWorkspaceSizeInBytes1 > 0:
    workspace1 = cp.cuda.alloc(extraWorkspaceSizeInBytes1)
    workspace_ptr1 = workspace1.ptr
else:
    workspace_ptr1 = 0

# 分配流2的工作区
if extraWorkspaceSizeInBytes2 > 0:
    workspace2 = cp.cuda.alloc(extraWorkspaceSizeInBytes2)
    workspace_ptr2 = workspace2.ptr
else:
    workspace_ptr2 = 0

# 在流1上应用量子门
cusv.apply_matrix(
    handle1,                         # cuStateVec 句柄1
    d_sv1.data.ptr,                  # 流1状态向量的设备指针
    cudtype.CUDA_C_64F,              # 状态向量的数据类型
    nIndexBits,                      # 量子比特数
    d_matrix.data.ptr,               # 矩阵的设备指针
    cudtype.CUDA_C_64F,              # 矩阵的数据类型
    cusv.MatrixLayout.ROW,           # 矩阵布局
    adjoint,                         # 是否为伴随矩阵
    targets,                         # 目标位
    len(targets),                    # 目标位数量
    controls,                        # 控制位
    0,                               # 控制位极性
    len(controls),                   # 控制位数量
    ctype.COMPUTE_64F,               # 计算类型
    workspace_ptr1,                  # 流1的工作区指针
    extraWorkspaceSizeInBytes1       # 流1的工作区大小
)

# 在流2上应用量子门
cusv.apply_matrix(
    handle2,                         # cuStateVec 句柄2
    d_sv2.data.ptr,                  # 流2状态向量的设备指针
    cudtype.CUDA_C_64F,              # 状态向量的数据类型
    nIndexBits,                      # 量子比特数
    d_matrix.data.ptr,               # 矩阵的设备指针
    cudtype.CUDA_C_64F,              # 矩阵的数据类型
    cusv.MatrixLayout.ROW,           # 矩阵布局
    adjoint,                         # 是否为伴随矩阵
    targets,                         # 目标位
    len(targets),                    # 目标位数量
    controls,                        # 控制位
    0,                               # 控制位极性
    len(controls),                   # 控制位数量
    ctype.COMPUTE_64F,               # 计算类型
    workspace_ptr2,                  # 流2的工作区指针
    extraWorkspaceSizeInBytes2       # 流2的工作区大小
)

# 等待两个流中的任务完成
stream1.synchronize()
stream2.synchronize()

# 销毁句柄，释放资源
cusv.destroy(handle1)
cusv.destroy(handle2)
